This notebook demonstrates a test of refining the fundamental matrix using the slopes between keypoints.

In [19]:
import os
import pandas as pd
import sys
sys.path.insert(0, os.path.abspath('..'))

import unittest

from autocnet.examples import get_path
from autocnet.fileio.io_controlnetwork import to_isis
from autocnet.fileio.io_controlnetwork import write_filelist
from autocnet.graph.network import CandidateGraph
from autocnet.fileio import utils

#%pylab inline
%pylab qt4

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [20]:
serial_numbers = {'AS15-M-0295_SML.png': '1971-07-31T01:24:11.754',
                'AS15-M-0296_SML.png': '1971-07-31T01:24:36.970',
                'AS15-M-0297_SML.png': '1971-07-31T01:25:02.243',
                'AS15-M-0298_SML.png': '1971-07-31T01:25:27.457',
                'AS15-M-0299_SML.png': '1971-07-31T01:25:52.669',
                'AS15-M-0300_SML.png': '1971-07-31T01:26:17.923'}

for k, v in serial_numbers.items():
    serial_numbers[k] = 'APOLLO15/METRIC/{}'.format(v)

In [21]:
adjacency = get_path('two_image_adjacency.json')
basepath = get_path('Apollo15')
cg = CandidateGraph.from_adjacency(adjacency, basepath=basepath)

cg.extract_features(method='sift', extractor_parameters={"nfeatures":500})
cg.match_features(k=2)

In [22]:
for source, destination, edge in cg.edges_iter(data=True):
    # Perform the symmetry check
    edge.symmetry_check()
    # Perform the ratio test
    edge.ratio_check(clean_keys=['symmetry'])

In [23]:
cg.apply_func_to_edges("compute_fundamental_matrix", clean_keys=['symmetry', 'ratio'])

In [24]:
f = cg.edge[0][1].fundamental_matrix

In [25]:
df=utils.calculate_slope(f.x1, f.x2)
len(df.values.ravel())

2000

In [26]:
f.mask[f.mask==True].equals(f.mask[f.mask])

True

In [69]:
nbr_inliers = 20
fp = np.array(np.random.standard_normal((nbr_inliers, 2)))  # inliers
static_F = np.array([[4, 0.5, 10], [0.25, 1, 5], [0.2, 0.1, 1]])
fph = np.hstack((fp, np.ones((nbr_inliers, 1))))
tp = static_F.dot(fph.T)
# normalize hom. coordinates
tp /= tp[-1, :np.newaxis]
df1 = pd.DataFrame(fp, columns=['x', 'y'])
df2 = pd.DataFrame(tp.T[:, :2], columns=['x', 'y'])

In [70]:
df1

,x,y
0,0.143192,2.192324
1,0.846174,0.197930
2,0.622564,-1.081280
3,0.881229,-0.587017
4,0.288810,1.927145
5,-0.460151,-1.205435
6,0.235280,-0.942149
7,-1.773901,0.774746
8,-1.045653,0.395082
9,-2.001308,-0.385638


In [71]:
df2

,x,y
0,9.351072,5.792364
1,11.340072,4.549493
2,11.756981,4.008679
3,11.839718,4.145958
4,9.691355,5.597344
5,9.596680,4.672853
6,10.988238,4.320418
7,4.554858,7.376936
8,7.243608,6.182330
9,3.211032,7.331114


In [72]:
mask = df1.x > 0
mask

0      True
1      True
2      True
3      True
4      True
5     False
6      True
7     False
8     False
9     False
10     True
11     True
12     True
13    False
14     True
15     True
16     True
17     True
18    False
19    False
Name: x, dtype: bool

In [73]:
df1[mask]

,x,y
0,0.143192,2.192324
1,0.846174,0.197930
2,0.622564,-1.081280
3,0.881229,-0.587017
4,0.288810,1.927145
6,0.235280,-0.942149
10,0.354249,-0.857275
11,0.140351,0.388420
12,0.784891,0.506238
14,0.203228,0.826552


In [74]:
df2[mask]

,x,y
0,9.351072,5.792364
1,11.340072,4.549493
2,11.756981,4.008679
3,11.839718,4.145958
4,9.691355,5.597344
6,10.988238,4.320418
10,11.154310,4.295189
11,10.081068,5.083368
12,11.090312,4.722136
14,9.993929,5.232222


In [75]:
df1[mask] = df2[mask]

In [76]:
df1

,x,y
0,9.351072,5.792364
1,11.340072,4.549493
2,11.756981,4.008679
3,11.839718,4.145958
4,9.691355,5.597344
5,-0.460151,-1.205435
6,10.988238,4.320418
7,-1.773901,0.774746
8,-1.045653,0.395082
9,-2.001308,-0.385638


In [16]:
f.refine(df=utils.calculate_slope(f.x1, f.x2))

In [ ]:
f.rollback()

In [ ]:
f.mask

In [ ]:
figsize(18,18)
fig, ax = plt.subplots(1,1)
ax = cg.edge[0][1].plot(clean_keys=['ratio', 'fundamental'], ax=ax)

In [ ]:
figsize(18,18)
fig, ax = plt.subplots(1,1)
ax = cg.edge[0][2].plot(clean_keys=['ratio', 'symmetry', 'fundamental', 'subpixel'], ax=ax)